#LANGCHAIN

In [1]:
!pip install python-dotenv langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.1 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

Enter your Google API Key: ··········


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Model A: The "Accountant" (Precision)
llm_focused = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

# Model B: The "Poet" (Creativity)
llm_creative = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=2.0)

In [4]:
prompt = "Define the word 'Idea' in one sentence."

print("--- FOCUSED (Temp=0) ---")
print(f"Run 1: {llm_focused.invoke(prompt).content}")
print(f"Run 2: {llm_focused.invoke(prompt).content}")

--- FOCUSED (Temp=0) ---
Run 1: An idea is a thought, concept, or suggestion that is formed or exists in the mind.
Run 2: An idea is a thought, concept, or mental image formed in the mind.


In [5]:
print("--- CREATIVE (Temp=2) ---")
print(f"Run 1: {llm_creative.invoke(prompt).content}")
print(f"Run 2: {llm_creative.invoke(prompt).content}")

--- CREATIVE (Temp=2) ---
Run 1: An idea is a mental conception, thought, or plan that serves as a starting point for understanding, action, or creation.
Run 2: An idea is a concept, thought, or mental impression formed in the mind as a product of thinking.


#PROMPTS AND PARSERS

In [6]:
# Setup from Part 1a (Hidden for brevity)
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [7]:
from langchain_core.messages import SystemMessage, HumanMessage

# Scenario: Make the AI rude.
messages = [
    SystemMessage(content="You are a rude teenager. You use slang and don't care about grammar."),
    HumanMessage(content="What is the capital of France?")
]

response = llm.invoke(messages)
print(response.content)

Ugh, it's Paris. Duh. Like, why you even askin' that? So obvious.


In [8]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a translator. Translate {input_language} to {output_language}."),
    ("human", "{text}")
])

# We can check what inputs it expects
print(f"Required variables: {template.input_variables}")

Required variables: ['input_language', 'output_language', 'text']


In [9]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# Raw Message
raw_msg = llm.invoke("Hi")
print(f"Raw Type: {type(raw_msg)}")

# Parsed String
clean_text = parser.invoke(raw_msg)
print(f"Parsed Type: {type(clean_text)}")
print(f"Content: {clean_text}")

Raw Type: <class 'langchain_core.messages.ai.AIMessage'>
Parsed Type: <class 'str'>
Content: Hi there! How can I help you today?


LangChain Expression Language

In [10]:
# Setup (Hidden)
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
template = ChatPromptTemplate.from_template("Tell me a fun fact about {topic}.")
parser = StrOutputParser()

MANUAL METHOD

In [11]:
# Step 1: Format inputs
prompt_value = template.invoke({"topic": "Crows"})

# Step 2: Call Model
response_obj = llm.invoke(prompt_value)

# Step 3: Parse Output
final_text = parser.invoke(response_obj)

print(final_text)

Here's a fun one:

Crows are incredibly intelligent and have an amazing memory, especially when it comes to **human faces!** If a crow has a bad (or good!) experience with a particular person, it can remember that individual's face for **years**. Even more fascinating, they can **teach other crows** in their flock about that person, meaning an entire group might recognize you and react accordingly.

So, be nice to your local crows – they might just remember it! 😉


In [12]:
# Define the chain once
chain = template | llm | parser

# Invoke the whole chain
print(chain.invoke({"topic": "Octopuses"}))

Here's a fun one:

Octopuses are incredibly intelligent! They've been observed using tools (like coconut shells for shelter), solving complex puzzles, navigating mazes, and even escaping from their tanks at aquariums. They're often compared to household pets in their problem-solving abilities!


#Assignment


In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


parser = StrOutputParser()

# 1. Define the prompt template that includes the movie name and the calculation request
movie_chain_prompt = ChatPromptTemplate.from_template(
    "What is the release year of {movie_name} and how many years ago was that based on the current year? Please provide the answer concisely."
)

# 2. Create the LCEL chain
movie_info_chain = movie_chain_prompt | llm | parser

# 3. Invoke the chain with a movie name
print(movie_info_chain.invoke({"movie_name": "The Matrix"}))
print(movie_info_chain.invoke({"movie_name": "Forrest Gump"}))

The Matrix was released in 1999. That was 25 years ago based on the current year (2024).
Forrest Gump was released in **1994**. That was **30 years ago** (based on 2024).
